In [69]:
# import statements
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn import cross_validation
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import plot    
import plotly.graph_objs as go
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import RMSprop
from sklearn.cross_validation import train_test_split
import keras
from random import randint

import random

# --- GENSIM PACKAGE ---
import gensim, logging
from gensim.models import Word2Vec, doc2vec, Doc2Vec

from gensim.models import doc2vec
from collections import namedtuple


%matplotlib inline


### function to get doc2vec

In [70]:
def doc_to_vec(doc):
    doc1 = []
    doc1.append(doc)
    docs = []
    analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
    for i, text in enumerate(doc1):
        words = text.lower().split()
        tags = [i]
        docs.append(analyzedDocument(words, tags))

    # Train model (set min_count = 1, if you want the model to work with the provided example data set)

    model = doc2vec.Doc2Vec(docs, size = 200, window = 300, min_count = 1, workers = 4)

    # Get the vectors

    s = model.docvecs
    return s

### loading data

In [71]:
train_v1 = pd.read_json('../data/train-v1.1.json')
test_v1 = pd.read_json('../data/dev-v1.1.json')
squad_train = pd.read_json('../data/squad_train_doc.json')
squad_test = pd.read_json('../data/squad_dev_doc.json')

In [72]:
data_train = squad_train
data_train.rename(columns={'passages': 'documents'}, inplace=True)


In [73]:
data_train.head()

,documents,title
0,"[{'context': 'Architecturally, the school has ...",University_of_Notre_Dame
1,[{'context': 'Beyoncé Giselle Knowles-Carter (...,Beyoncé
2,[{'context': 'Montana i/mɒnˈtænə/ is a state i...,Montana
3,"[{'context': 'The phrase ""in whole or in part""...",Genocide
4,[{'context': 'The emergence of resistance of b...,Antibiotics


### Document list

In [74]:
title_list = [data_train.title[i] for i in range(data_train.shape[0])]

### Segregating Dataframe

In [75]:
## Returns list of all context and list of all question sets

def get_compact_dataframe():
    context_list = []
    question_list = []
    
    for doc in data_train.documents:
        context_list.append(get_each_context_each_questionSet(doc)[0])
        question_list.append(get_each_context_each_questionSet(doc)[1])
    
    return context_list, question_list

In [76]:
## Pass a document and it returns list of context and list of set of questions

def get_each_context_each_questionSet(document):
    each_doc_context_list = [document[i]['context'] for i in range(len(document))]
    each_doc_question_list = [document[i]['questions'] for i in range(len(document))]
    return  each_doc_context_list, each_doc_question_list

In [77]:
context_list, question_list = get_compact_dataframe()[0], get_compact_dataframe()[1]
compact_dataframe = pd.DataFrame({'title':title_list, 'context':context_list, 'questions': question_list})

## DATAFRAME

In [78]:
compact_dataframe.head()

,context,questions,title
0,"[Architecturally, the school has a Catholic ch...",[[To whom did the Virgin Mary allegedly appear...,University_of_Notre_Dame
1,[Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ ...,"[[In what city did Beyonce grow up?, In what R...",Beyoncé
2,[Montana i/mɒnˈtænə/ is a state in the Western...,[[How many ranges are part of the Rocky Mounta...,Montana
3,"[The phrase ""in whole or in part"" has been sub...",[[What 2001 case was declared genocide by the ...,Genocide
4,[The emergence of resistance of bacteria to an...,"[[What does emergence of resistance reflect?, ...",Antibiotics


In [79]:
compact_dataframe[compact_dataframe.title == '2008_Sichuan_earthquake']

,context,questions,title
10,[The 2008 Sichuan earthquake or the Great Sich...,"[[What was the earthquake named?, What did the...",2008_Sichuan_earthquake


### Total questions

In [80]:
list_questions_perDOC = []
count = 0

for each_doc_question in compact_dataframe.questions:

    Doc_questions = []
    for each_Set in each_doc_question:
        for each_question in each_Set:
            Doc_questions.append(each_question)
            count +=1
    
    list_questions_perDOC.append(Doc_questions)

### Implementing Gensim 

In [81]:
class DocIterator(object):
    
    ## Initailizes document's list(doc1,doc2...) and its label's list('Book_1','Book_2',...)
    def __init__(self, doc_list, labels_list):
        self.doc_list = doc_list
        self.labels_list = labels_list
    
    ## Assigns label1 to a list of all words in doc1, label2 to all words in doc2, etc.
    def __iter__(self):
        
        for idx, doc in enumerate(self.doc_list):
            yield doc2vec.LabeledSentence(words=doc.split(), tags=[self.labels_list[idx]])

In [82]:
## Contains the whole data

data = []
docLabels = title_list

for i in docLabels:
    x = compact_dataframe[compact_dataframe.title == i].context.sum()
    y = ' '.join(x)
    data.append(y)

In [83]:
iter_docs = DocIterator(data, docLabels)

### Model

In [84]:
model = Doc2Vec(iter_docs, size=200, window=50, min_count=1, workers=11, alpha=0.025, min_alpha=0.025, dbow_words = 1, iter = 20)

In [184]:
model.docvecs[0]

array([-4.14300776,  3.66796184,  5.63129187,  2.55063319,  2.43406987,
        2.73234034, -1.13330567, -2.64857841, -3.18167591, -3.83612204,
        4.50962067, -4.69891834, -0.97023374,  0.18418452, -0.28791553,
       -3.7861855 , -6.51898479, -4.95454741, -8.00101089,  4.75595617,
       -3.6635735 ,  0.64372057,  2.45392919, -1.85304654,  7.06742477,
       -2.61267376,  5.75118017,  1.56268179,  0.21649441, -1.66226172,
        0.27283433, -3.52976537,  1.81762564, -3.68014979, -0.97476965,
        4.36508322,  5.38206959, -5.33248806, -1.56811774, -0.35706735,
       -1.18078005,  0.8236841 ,  0.88663453,  0.95708627,  5.61691999,
        2.3893187 ,  0.82356215, -0.92819583, -2.15672278, -3.96543622,
        0.36938497,  1.54507279,  0.86059797,  3.92167115, -1.16093707,
        2.31430173, -5.10472679,  2.19292021,  2.58656931, -1.66382718,
        3.60732961,  3.6866529 ,  0.4933086 , -0.22857243,  2.00314879,
        2.55336642,  0.27661416,  2.70353341, -3.38253975,  4.27

### creating dummy dataframe using five questions from 5 documents

In [86]:
questions_list = ['To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?','In what city did Beyonce grow up?',
                 'How many ranges are part of the Rocky Mountains?',
                  'What 2001 case was declared genocide by the International Criminal Tribune for the Former Yugoslavia (ICTY)?',
                 'What does emergence of resistance reflect?']

In [87]:
data_documents = []
for i in range(5):
    for j in range(5):
        data_documents.append(title_list[j])  

In [88]:
data_qusetions = []
for i in range(5):
    for j in range(5):
        data_qusetions.append(questions_list[i])

In [89]:
def scores():
    scores = []
    for i in range(25):
        scores.append(random.random())
    return scores    

In [90]:
s1 = scores()
s2 = scores()
s3 = scores()

In [91]:
y =[]
rank = []
for i in range(5):
    k = 2
    for j in range(5):
        if(i==j):
            y.append(1)
            rank.append(1)
        else:
            y.append(0)
            rank.append(k)
            k += 1
rank

[1, 2, 3, 4, 5, 2, 1, 3, 4, 5, 2, 3, 1, 4, 5, 2, 3, 4, 1, 5, 2, 3, 4, 5, 1]

In [92]:
questions_vs_documents = pd.DataFrame(
    {'questions': data_qusetions,
     'documents': data_documents,
     's1': s1,
     's2': s2,
     's3': s3,
     'y': y,
     'rank':rank
    })

In [93]:
questions_vs_documents

,documents,questions,rank,s1,s2,s3,y
0,University_of_Notre_Dame,To whom did the Virgin Mary allegedly appear i...,1,0.367883,0.202304,0.061476,1
1,Beyoncé,To whom did the Virgin Mary allegedly appear i...,2,0.007341,0.244544,0.457042,0
2,Montana,To whom did the Virgin Mary allegedly appear i...,3,0.394381,0.058156,0.664137,0
3,Genocide,To whom did the Virgin Mary allegedly appear i...,4,0.086811,0.256104,0.751386,0
4,Antibiotics,To whom did the Virgin Mary allegedly appear i...,5,0.359495,0.461835,0.447536,0
5,University_of_Notre_Dame,In what city did Beyonce grow up?,2,0.788579,0.636348,0.556743,0
6,Beyoncé,In what city did Beyonce grow up?,1,0.375269,0.985254,0.345774,1
7,Montana,In what city did Beyonce grow up?,3,0.459289,0.404665,0.896401,0
8,Genocide,In what city did Beyonce grow up?,4,0.937288,0.429144,0.166462,0
9,Antibiotics,In what city did Beyonce grow up?,5,0.454147,0.755869,0.774915,0


In [96]:
def input_question(question):
    df1 = questions_vs_documents[((questions_vs_documents['questions'] == question)& (questions_vs_documents['rank'] == 1))]
    df2 = questions_vs_documents[((questions_vs_documents['questions'] == question)& (questions_vs_documents['rank'] == max(rank)))]  
    df3 = questions_vs_documents[((questions_vs_documents['questions'] == question)& (questions_vs_documents['rank'] == randint(2,4)))] 
    new_df = df1.append(df2, ignore_index=True)
    new_df2 =new_df.append(df3, ignore_index=True)
    doc_vec_question = doc_to_vec(question)[0]
    for i in range(3):
        new_df2['questions'][i] = doc_vec_question
        new_df2['documents'][i] = model.docvecs[title_list.index(new_df2['documents'][i])]
    y_train = new_df2['y']
    x_train_scores = new_df2[['s1','s2','s3']] 
    new_df2.drop(['y','rank','s1','s2','s3'],axis = 1, inplace = True)
    x_train_questions = new_df2
    return x_train_questions,y_train,x_train_scores


In [97]:
#model.docvecs[title_list.index(new_df2['documents'][i]

SyntaxError: unexpected EOF while parsing (<ipython-input-97-cf664fb156a8>, line 1)

In [98]:
x_train_questions,y_train,x_train_scores = input_question('To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?')

ValueError: cannot set a row with mismatched columns

In [64]:
row_index = 1
x_train_questions['questions'] 
#x_train_questions

0    [-0.00149376, -0.00135826, 0.000248178, -0.001...
1    [-0.00149376, -0.00135826, 0.000248178, -0.001...
2    [-0.00149376, -0.00135826, 0.000248178, -0.001...
Name: questions, dtype: object

In [52]:
for i in range(3):
    x_train_questions['questions'][i] = doc_to_vec(question)[0]

NameError: name 'question' is not defined

In [27]:
train_x_array_questions = []
for each in x_train_questions.as_matrix():
    for nested in each:
        train_x_array_questions.append(nested)

train_x_array_questions = np.array(train_x_array_questions)

In [28]:
train_x_array_question_docs = train_x_array_questions.reshape(3,2,200)

In [29]:
train_x_array_question_docs.shape

(3, 2, 200)

In [30]:
train_y_train = np.array(y_train)

In [31]:
train_x_array_question_docs

array([[[ -3.40906560e-01,   2.92871803e-01,  -5.39408147e-01, ...,
          -4.39507389e+00,  -3.68476152e+00,  -1.70372546e+00],
        [ -1.49376167e-03,  -1.35825737e-03,   2.48177908e-04, ...,
          -1.82787026e-03,   1.84030843e-03,   1.03644165e-03]],

       [[ -3.92986870e+00,   2.73946118e+00,   3.27372886e-02, ...,
          -2.22616744e+00,  -4.55049467e+00,   3.15341711e+00],
        [ -1.49376167e-03,  -1.35825737e-03,   2.48177908e-04, ...,
          -1.82787026e-03,   1.84030843e-03,   1.03644165e-03]],

       [[ -3.49512339e+00,  -3.37478042e-01,   2.22073987e-01, ...,
          -9.11642265e+00,  -7.75338650e+00,  -9.48857784e-01],
        [ -1.49376167e-03,  -1.35825737e-03,   2.48177908e-04, ...,
          -1.82787026e-03,   1.84030843e-03,   1.03644165e-03]]], dtype=float32)

In [32]:
model_questions = Sequential()

In [33]:
model_questions.add(Dense(500, input_shape=(2,200)))
model_questions.add(Dense(600, activation='relu'))
model_questions.add(Flatten())
model_questions.add(Dense(1))

In [34]:
model_questions.compile(optimizer=RMSprop(),loss='binary_crossentropy',metrics = ['accuracy'])

In [35]:
model_questions.fit(train_x_array_question_docs, y_train, batch_size=1, epochs=5, validation_split = .2, verbose=2)

Train on 2 samples, validate on 1 samples
Epoch 1/5
0s - loss: 8.4867 - acc: 0.0000e+00 - val_loss: 1.0000e-07 - val_acc: 0.0000e+00
Epoch 2/5
0s - loss: 8.0590 - acc: 0.0000e+00 - val_loss: 1.0000e-07 - val_acc: 0.0000e+00
Epoch 3/5
0s - loss: 8.0590 - acc: 0.0000e+00 - val_loss: 1.0000e-07 - val_acc: 0.0000e+00
Epoch 4/5
0s - loss: 8.0590 - acc: 0.0000e+00 - val_loss: 1.0000e-07 - val_acc: 0.0000e+00
Epoch 5/5
0s - loss: 8.0590 - acc: 0.0000e+00 - val_loss: 1.0000e-07 - val_acc: 0.0000e+00


In [36]:
model_questions.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 2, 500)            100500    
_________________________________________________________________
dense_2 (Dense)              (None, 2, 600)            300600    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1200)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 1201      
Total params: 402,301
Trainable params: 402,301
Non-trainable params: 0
_________________________________________________________________


In [37]:
x_train_scores

,s1,s2,s3
0,0.462008,0.370907,0.783126
1,0.634431,0.398488,0.242621
2,0.428439,0.554364,0.373254


In [38]:
train_x_train_scores = np.array(x_train_scores)

In [39]:
train_x_train_scores.shape

(3, 3)

In [40]:
model_scores = Sequential()

In [41]:
model_scores.add(Dense(500, input_shape=(3,)))
model_scores.add(Dense(600, activation='relu'))
model_scores.add(Dense(1))

In [42]:
model_scores.compile(optimizer=RMSprop(),loss='binary_crossentropy',metrics = ['accuracy'])

In [43]:
model_scores.fit(train_x_train_scores, y_train, batch_size=1, epochs=5, validation_split = .2, verbose=2)

Train on 2 samples, validate on 1 samples
Epoch 1/5
0s - loss: 8.0590 - acc: 0.5000 - val_loss: 1.0000e-07 - val_acc: 1.0000
Epoch 2/5
0s - loss: 8.0590 - acc: 0.5000 - val_loss: 1.0000e-07 - val_acc: 1.0000
Epoch 3/5
0s - loss: 8.0590 - acc: 0.5000 - val_loss: 1.0000e-07 - val_acc: 1.0000
Epoch 4/5
0s - loss: 8.0590 - acc: 0.5000 - val_loss: 1.0000e-07 - val_acc: 1.0000
Epoch 5/5
0s - loss: 8.0590 - acc: 0.5000 - val_loss: 1.0000e-07 - val_acc: 1.0000


In [44]:
from keras.layers import Dense, merge

In [45]:
model_merge = Sequential()
model_merge.add(merge([model_questions,model_scores], mode = 'concat'))
model_merge.add(Dense(1, init = 'normal', activation = 'relu'))
#sgd = SGD(lr = 0.1, momentum = 0.9, decay = 0, nesterov = False)
model_merge.compile(loss = 'binary_crossentropy', optimizer = RMSprop(), metrics = ['accuracy'])
#seed(2017)
model_merge.fit([train_x_array_question_docs, train_x_train_scores], y_train,batch_size=1, epochs=5, validation_split = .2, verbose=2)


/home/user/classification/lib/python3.5/site-packages/ipykernel/__main__.py:2: UserWarning:

The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.

/home/user/classification/lib/python3.5/site-packages/keras/legacy/layers.py:469: UserWarning:

The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.



AttributeError: 'Sequential' object has no attribute 'get_shape'

In [149]:
questions_vs_documents2 = pd.DataFrame(
    {'questions': data_qusetions,
     'documents': data_documents
    })

In [150]:
questions_vs_documents2['questions'][0]

'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'

In [151]:
questions_vs_documents3 = questions_vs_documents2[(questions_vs_documents['questions'] == 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?')]

In [152]:
questions_vs_documents3

,documents,questions
0,University_of_Notre_Dame,To whom did the Virgin Mary allegedly appear i...
1,Beyoncé,To whom did the Virgin Mary allegedly appear i...
2,Montana,To whom did the Virgin Mary allegedly appear i...
3,Genocide,To whom did the Virgin Mary allegedly appear i...
4,Antibiotics,To whom did the Virgin Mary allegedly appear i...


In [153]:
pk = doc_to_vec('To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?')[0]

In [154]:
questions_vs_documents3.questions.values 

array(['To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
       'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
       'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
       'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
       'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'], dtype=object)

In [160]:
questions_vs_documents3['questions'] = questions_vs_documents3.questions.replace(questions_vs_documents3.questions.values,[pk,pk,pk,pk,pk])

/home/user/classification/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [161]:
questions_vs_documents3

,documents,questions
0,University_of_Notre_Dame,-0.001494
1,Beyoncé,-0.001358
2,Montana,0.000248
3,Genocide,-0.001865
4,Antibiotics,-0.001031


In [142]:
if questions_vs_documents3[questions_vs_documents3.questions == 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?']:
    questions_vs_documents3.questions = pk

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [137]:
z

,documents,questions
0,University_of_Notre_Dame,To whom did the Virgin Mary allegedly appear i...
1,Beyoncé,To whom did the Virgin Mary allegedly appear i...
2,Montana,To whom did the Virgin Mary allegedly appear i...
3,Genocide,To whom did the Virgin Mary allegedly appear i...
4,Antibiotics,To whom did the Virgin Mary allegedly appear i...


In [136]:
z.questions = pk

ValueError: Length of values does not match length of index

In [127]:
values23

array([[ -1.49376167e-03,  -1.35825737e-03,   2.48177908e-04,
         -1.86451187e-03,  -1.03108771e-03,  -1.78812596e-03,
         -6.75766671e-04,   2.13008164e-03,   2.45031947e-03,
          1.43686181e-03,   2.32396601e-03,   1.66668533e-03,
          7.27022416e-04,  -7.36753063e-05,  -1.99432648e-03,
          1.47048582e-03,  -1.34569849e-03,   6.66542794e-04,
          4.38582414e-04,  -9.92912217e-04,   1.29773514e-03,
         -2.12226342e-03,  -1.34737708e-03,   2.18559941e-03,
          2.36536958e-03,  -1.05773041e-03,  -1.22474821e-03,
         -6.71010406e-04,   9.73643269e-04,  -2.38315016e-03,
          1.52538810e-03,   8.21877911e-04,  -1.21718680e-03,
         -1.48794171e-03,  -9.51257942e-04,  -2.44513620e-03,
         -2.32611527e-03,  -2.36360170e-03,   4.98520210e-04,
          1.21345697e-03,  -2.18996988e-03,  -2.10899953e-03,
         -2.14571063e-03,  -1.35921501e-03,  -3.40325525e-04,
         -4.87815065e-04,   7.45294208e-04,   2.92327983e-04,
        

In [121]:
questions_vs_documents3.questions.values = values23

AttributeError: can't set attribute

In [162]:
from gensim.models import doc2vec
from collections import namedtuple

# Load data

doc1 = ["This is a sentence", "This is another sentence"]

# Transform data (you can add more data preprocessing steps) 



In [178]:
def doc_2_vec(doc1):
    docs = []
    analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
    for i, text in enumerate(doc1):
        words = text.lower().split()
        tags = [i]
        docs.append(analyzedDocument(words, tags))

    # Train model (set min_count = 1, if you want the model to work with the provided example data set)

    model = doc2vec.Doc2Vec(docs, size = 100, window = 300, min_count = 1, workers = 4)
    s = list(model.docvecs)    
    return s

In [179]:
pqqq = doc_2_vec(doc1)

In [180]:
pqqq

[array([ -2.97993422e-03,  -2.70987814e-03,   4.90625855e-04,
         -3.72301368e-03,  -2.05913745e-03,  -3.57176247e-03,
         -1.34426728e-03,   4.25264239e-03,   4.89825802e-03,
          2.86883628e-03,   4.64429846e-03,   3.32708028e-03,
          1.44610624e-03,  -1.45733837e-04,  -3.99222691e-03,
          2.93396157e-03,  -2.68466794e-03,   1.33024855e-03,
          8.75272905e-04,  -1.98375899e-03,   2.59350357e-03,
         -4.23856592e-03,  -2.69876397e-03,   4.36984375e-03,
          4.72765882e-03,  -2.11208034e-03,  -2.44789454e-03,
         -1.33529457e-03,   1.94521772e-03,  -4.76235431e-03,
          3.05004814e-03,   1.64397852e-03,  -2.42934912e-03,
         -2.96619488e-03,  -1.89820828e-03,  -4.88210516e-03,
         -4.63751750e-03,  -4.72059241e-03,   9.94084287e-04,
          2.42144475e-03,  -4.37540142e-03,  -4.21283999e-03,
         -4.28563217e-03,  -2.71985563e-03,  -6.79032644e-04,
         -9.69701621e-04,   1.48667546e-03,   5.78977051e-04,
        